In [ ]:
import matplotlib.pyplot as plt
from utilities import *
import numpy as np

In [ ]:
### Get an overview of the histograms calculated.

soc_id3 = run_sql('soc_requests')
soc_id3['time'] = pd.to_datetime(soc_id3['time'])
soc_id3.set_index(['vehicle_id', 'time'], inplace=True)
soc_id3

In [ ]:
soc_cupras = run_sql('cuprasoc')
def charging(x): return np.mean(x) > 0 
soc_cupras['time'] = pd.to_datetime(soc_cupras['time'], utc=True)
soc_cupras.set_index(['vehicle_id', 'time'], inplace=True)
# rescale linearly to BMS SOC
soc_cupras = (soc_cupras * 0.92 + 6)
soc_cupras

In [ ]:
import seaborn as sns 
sns.lineplot(data=soc_id3.reset_index(), x='time', y='soc', hue='vehicle_id')
import pandas as pd
import matplotlib.pyplot as plt

# --- assume your SQL result is already in a DataFrame called `df` ---
# If not, replace the next line with the variable you’re using.
df = df.copy()

# 1⃣  Ensure proper ordering
df = df.sort_index(level=['vehicle_id', 'time'])

# 2⃣  Compute the forward time-difference inside each vehicle
dt = (                          # timedelta (with +1 row shift)
    df.index.get_level_values('time')
       .to_series()
       .groupby(df.index.get_level_values('vehicle_id'))
       .shift(-1)                      # next timestamp
    - df.index.get_level_values('time')
)
df['time_diff_sec'] = dt.dt.total_seconds()

# 3⃣  Keep only rows that have a valid, positive gap
df_w = df[df['time_diff_sec'].notna() & (df['time_diff_sec'] > 0)]

# 4⃣  Histogram of SOC weighted by the gap length
plt.figure(figsize=(7,4))
plt.hist(
    df_w['soc'],
    bins=20,                      # tweak if you need finer/coarser bins
    weights=df_w['time_diff_sec']
)
plt.xlabel('State of Charge (%)')
plt.ylabel('Time-weighted count (seconds)')
plt.title('SOC distribution weighted by time between readings')
plt.tight_layout()
plt.show()

sns.lineplot(data=soc_cupras.reset_index(), x='time', y='soc', hue='vehicle_id')


In [ ]:
soc = pd.concat([soc_id3, soc_cupras]).sort_index()

soc['diff'] = soc.sort_index().groupby('vehicle_id')['soc'].diff()
soc['prev_soc'] = soc.groupby('vehicle_id').soc.shift(1)

In [ ]:
soc['cycle'] = (soc['diff'] > 0).astype(int).diff().abs().cumsum().fillna(0)
soc = soc.join(soc.groupby('cycle')['diff'].sum().rename('cycle_delta'), on='cycle')
soc
#(abs(soc['cycle_delta']) > 1.5).astype(int).diff().apply(lambda x: max(0,x)).cumsum()

In [ ]:
sign = -1
oldcycle = 0
metacycle = 0
metacycles = []

for i, row in soc.iterrows():
    cycle = row['cycle']
    if cycle != oldcycle:
        if abs(row['cycle_delta']) > 2:
            metacycle +=1 
    oldcycle = cycle
    metacycles = metacycles + [metacycle]

soc['metacycle'] = metacycles

soc.rename(columns={'cycle':'subcycle', 'metacycle':'cycle'}, inplace=True)

In [ ]:
def charging(x): return np.mean(x) > 0 

cycles = soc.reset_index(level=1).groupby(['vehicle_id','cycle']).agg(
    {'time':['min','max'], 'soc':['min','max'], 'prev_soc':['min','max'], 'diff': [charging, 'sum']})
cycles.columns = cycles.columns.map('_'.join)
cycles['soc_min'] = cycles.apply(lambda x: min(x.soc_min, x.prev_soc_min), axis=1)
cycles['soc_max'] = cycles.apply(lambda x: max(x.soc_max, x.prev_soc_max), axis=1)

cycles['DOD'] = (cycles['soc_max'] - cycles['soc_min']) * ((-1)**cycles['diff_charging'].astype(int))
cycles['dt'] = (cycles['time_max'] - cycles['time_min']) / np.timedelta64(1,'h')

cycles = cycles.drop(
    columns=['prev_soc_max', 'prev_soc_min', 'diff_sum', 'diff_charging'])

cycles.to_csv('../export/DOD_cycles.csv')

In [ ]:
cycles.DOD.plot.hist(bins=45)
plt.savefig('../export/DOD_bins.png')

In [ ]:
fig, ax = plt.subplots(1,2)
# discharging
cycles.loc[cycles.DOD > 0].soc_min.plot.hist(ax=ax[0],title='SOC after Driving', bins=50)
# charging
cycles.loc[cycles.DOD < 0].soc_max.plot.hist(ax=ax[1],title='SOC after Charging', bins=50)

ax[0].set_ylim(0,550)
ax[1].set_ylim(0,550)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# --- assume your SQL result is already in a DataFrame called `df` ---
# If not, replace the next line with the variable you’re using.
df = soc.copy()

# --- assume df already has a MultiIndex (vehicle_id, time) -------------
idx       = df.index                      # current MultiIndex
vehicle   = idx.get_level_values('vehicle_id')

# 🪄 1-liner that handles mixed localisation
time_utc  = pd.to_datetime(idx.get_level_values('time'), utc=True)

# re-assemble the MultiIndex with the patched time level
df.index  = pd.MultiIndex.from_arrays([vehicle, time_utc],
                                      names=idx.names)


# 1 Ensure proper ordering
df = df.sort_index(level=['vehicle_id', 'time'])

import pandas as pd
import matplotlib.pyplot as plt

# 0 ️⃣  df already uses a MultiIndex  (vehicle_id, time)
# ----- compute forward gap in seconds -------------------------------
ts        = pd.Series(df.index.get_level_values('time'),  # ← values
                      index=df.index,                     # ← keep index!
                      name='time')

next_ts   = ts.groupby(df.index.get_level_values('vehicle_id')).shift(-1)
df['time_diff_sec'] = (next_ts - ts).dt.total_seconds()

# drop rows whose gap is too large (weeks)
df_w = df[df['time_diff_sec'].lt(14*24*60*60) ]

# ----- time-weighted SOC histogram ----------------------------------
plt.figure(figsize=(7,4))
plt.hist(df_w['soc'],
         bins=50,
         weights=df_w['time_diff_sec'])
plt.xlabel('State of Charge (%)')
plt.ylabel('Time-weighted count (seconds)')
plt.title('SOC distribution weighted by time between readings')
plt.tight_layout()
plt.show()


In [ ]:
soc[['soc', 'diff']].to_csv('../export/soc_cycles.csv')